In [1]:
import sys
# !{sys.executable} -m spacy download en
import re, numpy as np, pandas as pd
from pprint import pprint

# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
import nltk
import glob

In [2]:
nltk.download('stopwords')
en_stop = list(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to C:\Users\Dell
[nltk_data]     Inspiron\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
stopwords_new = ('h','−','given','p','z','w','-','use','selection','also','guess', 'price', 'house', 'repeating', 'four', 'n', 'times', 'create', 'n', 'finalized', 'let', 'say','bigger', 'picture', 'main', 'methods', 'avoid','go','together','several','com', 'built','similar','select','number','three','point','points','clear','well','shown', 'depicts','making', 'take', 'entire','accepted', 'salary', 'greater', 'commute', 'less', 'hour', 'incentives', 'offered', 'build','input','repeat','every', 'branch','estimating','done','commonly','uses', 'known', 'labeled','q', 'differences','value','steps','must','offers','offer','calculate','first','consider','usually','www','want','–','split','set','using','values','time','interview','mean',"example","means","used","two","one","k","•","λ","sure","like","hi","yeah","um","et","al","https","http","version","org","text","x1","human","x","b","n,""f","-","c","e","g","da","is","a", "try","think", "maybe", 
                 "actually", "everything", "right","really","share","today","going", "would","try","action","setting","meetings",
                "thing","sometimes","image", "meeting","label", "model", "working", "basically", "start", "problem", "mandy", "yesterday",
                 "function", "great", "speaker","minute","update","running", "monday","thought", "sense","based","leave","sophia","nice",
                 "good", "work", "stuff","project","kind","guy","interesting","kind","data","user","screen",
                 "leaving","view", "idea")
for i in stopwords_new:
    en_stop.append(i)

In [4]:
# keywords_one=[]
with open('stopwords.txt', encoding="utf8") as f:
    for line in f.readlines():
        line = str(line).replace("\n","")
#         line = str(line).replace("\ufeff","")
#         line=line.encode('utf-8', errors='ignore')
        en_stop.append(line)

In [5]:
docs=[]
str2=[]
for file_name in glob.iglob('C://Users//Dell Inspiron//Downloads//Transcript//*', recursive=True): 
    with open(file_name) as f:
        dict_cont_org=''
        for line in f.readlines():
            name=line.split(':')
#             print(name[1])
            dict_cont_org=dict_cont_org+name[1]
        docs.append(dict_cont_org)

In [6]:
for i in docs:
    i=i.replace("\n"," ")

In [7]:
import spacy
spacy.load('en_core_web_sm')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [8]:
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)
from nltk.stem import PorterStemmer
def get_stem(word):
#     p=PorterStemmer()
    return PorterStemmer().stem(word)

In [9]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 3]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma2(token) for token in tokens]
    tokens = [token for token in tokens if token not in en_stop]
#     tokens = [get_stem(token) for token in tokens]
    return tokens

In [10]:
import random
text_data = []

In [20]:
text_data=[]
str2=[]
for doc in docs:
        tokens = prepare_text_for_lda(doc)
        if random.random() > .90:
            print(tokens)
            text_data.append(tokens)

['francis', 'danish', 'heavily', 'dish', 'restaurant', 'packing', 'lord', 'fence', 'moving', 'moving', 'sign', 'trite', 'zoom', 'kahlo', 'visit', 'hear', 'francis', 'finish', 'living', 'living', 'fellowship', 'large', 'docker', 'cheesy', 'game', 'manage', 'gnash', 'adam', 'team', 'member', 'younes', 'team', 'member', 'day', 'absolutely', 'task', 'needed', 'hear', 'eventually', 'task', 'asked', 'looked', 'type', 'wear', 'people', 'independently', 'baby', 'person', 'wanted', 'interested', 'streaking', 'dating', 'force', 'test', 'talking', 'telling', 'team', 'absolutely', 'dinitia', 'fortunately', 'resolved', 'creating', 'area', 'latent', 'presentation', 'apply', 'verity', 'answered', 'humidity', 'fact', 'bunch', 'code', 'closed', 'wanted', 'venue', 'save', 'notebook', 'reload', 'safe', 'deciding', 'notice', 'thinking', 'loaded', 'load', 'changed', 'caused', 'issue', 'loaded', 'loaded', 'true', 'assumes', 'form', 'format', 'tree', 'network', 'loading', 'transposing', 'dimension', 'issue',

In [21]:
data_words=text_data
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
texts = [bigram_mod[doc] for doc in data_words]

In [22]:
# def process_words(texts, stop_words=en_stop, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
#     """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
#     texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
#     texts = [bigram_mod[doc] for doc in texts]
#     texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
#     texts_out = []
#     nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
#     for sent in texts:
#         doc = nlp(" ".join(sent)) 
#         texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
#     # remove stopwords once more after lemmatization
#     texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
#     return texts_out

# data_ready = process_words(data_words)  # processed Text Data!

In [26]:
# Create Dictionary
data_ready=texts
id2word = corpora.Dictionary(data_ready)

# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_ready]

# Build LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 4, id2word=id2word, passes=20)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=25)
for topic in topics:
    print("Topic "+ str(topic[0])+ "    "+topic[1]+"\n")

Topic 0    0.012*"presentation" + 0.008*"fine" + 0.008*"combine" + 0.006*"hear" + 0.006*"people" + 0.006*"issue" + 0.006*"living" + 0.006*"fellowship" + 0.006*"francis" + 0.006*"loaded" + 0.006*"creating" + 0.006*"team" + 0.006*"needed" + 0.004*"docker" + 0.004*"code" + 0.004*"score" + 0.004*"court" + 0.004*"wanted" + 0.004*"wanna" + 0.004*"absolutely" + 0.004*"food" + 0.004*"size" + 0.004*"save" + 0.004*"finish" + 0.004*"moving"

Topic 1    0.020*"docker" + 0.017*"class" + 0.014*"year" + 0.014*"response" + 0.011*"correct" + 0.009*"grab" + 0.009*"splash" + 0.007*"watchman" + 0.006*"tomorrow" + 0.006*"note" + 0.006*"title" + 0.006*"shell" + 0.006*"attribute" + 0.006*"product" + 0.006*"website" + 0.005*"common" + 0.005*"dave" + 0.004*"pretty" + 0.004*"connect" + 0.004*"court" + 0.004*"called" + 0.004*"thinking" + 0.004*"multiple" + 0.004*"beautiful" + 0.004*"cool"

Topic 2    0.025*"test" + 0.014*"choice" + 0.014*"change" + 0.014*"host" + 0.014*"local" + 0.012*"a.b." + 0.012*"showing" + 

In [27]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary=ldamodel.id2word)
vis

F:\Anaconda\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.134480  0.021887       1        1  55.052429
3      0.083565  0.092677       2        1  16.109734
0      0.040278 -0.120866       3        1  15.936504
2      0.010637  0.006302       4        1  12.901328, topic_info=    Category       Freq        Term      Total  loglift  logprob
196  Default   7.000000        test   7.000000  30.0000  30.0000
275  Default   6.000000      reward   6.000000  29.0000  29.0000
296  Default  13.000000     website  13.000000  28.0000  28.0000
223  Default   7.000000       bacon   7.000000  27.0000  27.0000
231  Default   6.000000       click   6.000000  26.0000  26.0000
..       ...        ...         ...        ...      ...      ...
316   Topic4   1.113463       close   4.815943   0.5834  -5.3098
235   Topic4   1.114165    decision   5.403700   0.4689  -5.3091
37    Topic4   1.113965        code   7.818373   0.0993  -5.3093
363   Topic4   1.113946  javascript   5.404052   0.4686  -5.3093
234   Topic4   1.113588        cool   6.172803   0.3353  -5.3097

[212 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0         1  0.522545        a.b.
0         3  0.130636        a.b.
0         4  0.391908        a.b.
1         1  0.604543  absolutely
1         3  0.302271  absolutely
...     ...       ...         ...
525       2  0.721528      worked
416       4  0.913368       worse
218       1  0.893602        year
218       3  0.068739        year
218       4  0.068739        year

[245 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 1, 3])

In [16]:
test_docs=[]
str2=[]
for file_name in glob.iglob('C://Users//Dell Inspiron//Downloads//test_trans//*', recursive=True): 
    with open(file_name) as f:
        test_cont_org=''
        for line in f.readlines():
            name=line.split(':')
#             print(name[1])
            test_cont_org=dict_cont_org+name[1]
        test_docs.append(dict_cont_org)

In [17]:
for i in test_docs:
    i=i.replace("\n"," ")

In [28]:
testtext_data=[]
str2=[]
for doc in test_docs:
        t_tokens = prepare_text_for_lda(doc)
        if random.random() > .60:
            print(t_tokens)
            testtext_data.append(t_tokens)

['grab', 'perfect', 'small', 'doubt', 'connor', 'parsing', 'parse', 'parsing', 'grease', 'parcel', 'adapt', 'called', 'asked', 'order', 'political', 'dave', 'ally', 'iptv', 'complete', 'city', 'beautifui', 'specification', 'choose', 'class', 'unique', 'search', 'easily', 'close', 'money', 'closer', 'belittle', 'clothes', 'clothes', 'diarrhea', 'album', 'scenting', 'ray', 'finale', 'fierce', 'girl', 'string', 'box', 'kenny', 'left', 'spro', 'offiah', 'screw', 'distinct', 'tube', 'class', 'early', 'light', 'expected', 'future', 'blah', 'blah', 'bull', 'material', 'fitting', 'rising', 'list', 'jealous', 'scrapped', 'trip', 'channel', 'coming', 'month', 'saudi', 'thes', 'tour', 'essentially', 'weapon', 'ignore', 'life', 'swiss', 'unique', 'year', 'early', 'gravel', 'elba', 'grabbing', 'care', 'grab', 'essentially', 'deal', 'object', 'mark', 'wright', 'object', 'element', 'release', 'object', 'correspondent', 'object', 'material', 'skipping', 'size', 'position', 'dark', 'place', 'position',

In [29]:
for doc in test_docs:
        t_tokens = prepare_text_for_lda(doc)
        bow_vector = id2word.doc2bow(t_tokens)
        res = ldamodel[bow_vector]
        for index, score in sorted(res, key=lambda tup: -1*tup[1]):
            print("Score: {}\t Topic: {}".format(score, ldamodel.print_topic(index, 4)))

Score: 0.7591813206672668	 Topic: 0.020*"docker" + 0.017*"class" + 0.014*"year" + 0.014*"response"
Score: 0.0946977362036705	 Topic: 0.012*"presentation" + 0.008*"fine" + 0.008*"combine" + 0.006*"hear"
Score: 0.07987362146377563	 Topic: 0.025*"test" + 0.014*"choice" + 0.014*"change" + 0.014*"host"
Score: 0.06624732911586761	 Topic: 0.020*"reward" + 0.020*"website" + 0.016*"button" + 0.016*"bacon"
Score: 0.7591842412948608	 Topic: 0.020*"docker" + 0.017*"class" + 0.014*"year" + 0.014*"response"
Score: 0.09468652307987213	 Topic: 0.012*"presentation" + 0.008*"fine" + 0.008*"combine" + 0.006*"hear"
Score: 0.07987121492624283	 Topic: 0.025*"test" + 0.014*"choice" + 0.014*"change" + 0.014*"host"
Score: 0.06625805050134659	 Topic: 0.020*"reward" + 0.020*"website" + 0.016*"button" + 0.016*"bacon"
